In [45]:
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets

from IPython.display import display

In [46]:
NUM_STATES = 4
NUM_OBSERVATIONS = 2
NUM_BELIEFS = 2 # TARGET vs. NON TARGET
ANGLE_RANGE = 2*np.pi

STATE_STEP = ANGLE_RANGE/NUM_STATES
STATE_CENTERS = np.arange(0, ANGLE_RANGE, STATE_STEP) + STATE_STEP
ANGL_DIST_BETWEEN_STATES = (STATE_CENTERS[np.newaxis, :] - STATE_CENTERS[:, np.newaxis])

PRIOR_TARGET_PROB = np.array([1/NUM_BELIEFS, 1-(1/NUM_BELIEFS)])
PRIOR_STATE_PROB = np.full(NUM_STATES, 1/NUM_STATES)
LIKELIHOOD_OBS_FROM_NONTARGET = 1/NUM_OBSERVATIONS

In [47]:
def get_state_transition_matrix(candidate_step):
    if candidate_step>0:
        mod_basis = 2*np.pi
    else:
        mod_basis = -2*np.pi

    angular_dist_between_states_in_extent = ANGL_DIST_BETWEEN_STATES % mod_basis
    angular_dist_after_action = (angular_dist_between_states_in_extent - candidate_step)
    angular_dist_after_action_in_extent = angular_dist_after_action % mod_basis
    
    sigma = STATE_STEP/2
    magnitude = 1 / np.sqrt((2*np.pi) * (sigma**2))
    state_transition = magnitude * np.exp(-0.5*((angular_dist_after_action_in_extent/sigma)**2))
    state_transition_normed = state_transition / state_transition.sum(axis=1)
    return state_transition_normed

In [48]:
candidate_step = np.pi/2
state_transition_matrix = get_state_transition_matrix(candidate_step)
state_transition_matrix.round(3)

array([[0.   , 0.881, 0.119, 0.   ],
       [0.   , 0.   , 0.881, 0.119],
       [0.119, 0.   , 0.   , 0.881],
       [0.881, 0.119, 0.   , 0.   ]])

In [49]:
candidate_step = -np.pi/2
state_transition_matrix = get_state_transition_matrix(candidate_step)
state_transition_matrix.round(3)

array([[0.   , 0.   , 0.119, 0.881],
       [0.881, 0.   , 0.   , 0.119],
       [0.119, 0.881, 0.   , 0.   ],
       [0.   , 0.119, 0.881, 0.   ]])

In [50]:
def observe_angle_state_pair(initial_angle):
   plt.figure(figsize=(6, 6))
   radius = 1
   radius_until = 5
   theta_obs1 = np.linspace(np.pi/3, 2*np.pi/3, 10)
   x_inner = radius * np.cos(theta_obs1)
   y_inner =  radius * np.sin(theta_obs1)
   x_poly = np.concatenate([radius_until*x_inner, x_inner[::-1]])
   y_poly = np.concatenate([radius_until*y_inner, y_inner[::-1]])
   plt.fill(x_poly, y_poly, facecolor='blue', alpha=0.25, linewidth=0)
   plt.plot(x_inner, y_inner, color='blue', linewidth=1)

   theta_obs0 = np.linspace(np.pi/3, -4*np.pi/3, 50)
   x_inner = radius * np.cos(theta_obs0)
   y_inner =  radius * np.sin(theta_obs0)
   x_poly = np.concatenate([radius_until*x_inner, x_inner[::-1]])
   y_poly = np.concatenate([radius_until*y_inner, y_inner[::-1]])
   plt.fill(x_poly, y_poly, facecolor='orange', alpha=0.25, linewidth=0)
   plt.plot(x_inner, y_inner, color='orange', linewidth=1)

   for theta in STATE_CENTERS:
      r_outer = 4 * radius_until * radius
      plt.plot([radius*np.cos(theta), r_outer*np.cos(theta)], 
               [radius*np.sin(theta), r_outer*np.sin(theta)],
                  color='k', linewidth=1.5, solid_capstyle='round', zorder=1)
      plt.plot([radius*np.cos(theta + (STATE_STEP/2)), r_outer*np.cos(theta + (STATE_STEP/2))], 
               [radius*np.sin(theta + (STATE_STEP/2)), r_outer*np.sin(theta + (STATE_STEP/2))],
                  color='k', linewidth=1, linestyle='dashed', zorder=1)
      
   if initial_angle >= 60 and initial_angle <= 120:
      code_received = 1
   else:
      code_received = 0
   initial_radians = np.radians(initial_angle)
   initial_state = int((initial_radians + (STATE_STEP/2)) / STATE_STEP)
   plt.scatter(radius_until*radius*np.cos(initial_radians), radius_until*radius*np.sin(initial_radians),
            s=200, marker='.', facecolor='r', edgecolor='k', zorder=4, label=f'State={initial_state}\nCode={code_received}')

   plt.gca().set_aspect('equal', adjustable='box')
   plt.xticks([])
   plt.yticks([])
   GRID_EXTENT = 7
   plt.xlim(-GRID_EXTENT, GRID_EXTENT)
   plt.ylim(-GRID_EXTENT, GRID_EXTENT)
   plt.legend()
   plt.show()


In [51]:
initial_angle_slider = widgets.IntSlider(
    value=0,  
    min=0,
    max=360,
    description="Initial Angle",
    style={'description_width': 'initial'},
    continuous_update=True,
    layout=widgets.Layout(width="600px"),
)

interactive_plot = widgets.interactive(
    observe_angle_state_pair, initial_angle=initial_angle_slider)
display(interactive_plot)

interactive(children=(IntSlider(value=0, description='Initial Angle', layout=Layout(width='600px'), max=360, s…

In [52]:
code0_extent_in_states = np.array([np.pi/3, np.pi/3, np.pi/2, np.pi/2])
code1_extent_in_states = np.array([np.pi/6, np.pi/6,       0,       0])

prob_code0_in_states = code0_extent_in_states / STATE_STEP
prob_code1_in_states = code1_extent_in_states / STATE_STEP

EMISSION_MATRIX_B = np.vstack([prob_code0_in_states, prob_code1_in_states])
EMISSION_MATRIX_B

array([[0.66666667, 0.66666667, 1.        , 1.        ],
       [0.33333333, 0.33333333, 0.        , 0.        ]])

In [53]:
initial_angle = 0

if initial_angle >= 60 and initial_angle <= 120:
    code_received = 1
else:
    code_received = 0
initial_angle, code_received

(0, 0)

In [54]:
joint_prob_obs_state_t1 = PRIOR_STATE_PROB * EMISSION_MATRIX_B

joint_prob_obs0_state_t1 = joint_prob_obs_state_t1[code_received]
likelihood_obs0_from_target_t1 = joint_prob_obs0_state_t1.sum()
likelihood_obs0_t1 = np.array([likelihood_obs0_from_target_t1, LIKELIHOOD_OBS_FROM_NONTARGET])

posterior_t1 = (PRIOR_TARGET_PROB*likelihood_obs0_t1) / (PRIOR_TARGET_PROB*likelihood_obs0_t1).sum()
current_entropyS_t1 = (-posterior_t1*np.log2(posterior_t1)).sum()
current_entropyS_t1

np.float64(0.954434002924965)

In [55]:
candidate_steps = np.array([-np.pi/2, np.pi/2])
expected_forecast_t2_S_per_step = np.zeros(candidate_steps.shape[0])

In [56]:
for i, forecast_t2_candidate_step in enumerate(candidate_steps):
    state_transition_matrix = get_state_transition_matrix(forecast_t2_candidate_step)

    joint_prob_obs_state_forecast_t2 = (joint_prob_obs0_state_t1 @ state_transition_matrix) * EMISSION_MATRIX_B
    likelihood_obs_from_target_forecast_t2 = joint_prob_obs_state_forecast_t2.sum(axis=1)
    likelihood_obs_from_target_forecast_t2_given_obs0_at_t1 = likelihood_obs_from_target_forecast_t2 / likelihood_obs0_from_target_t1

    likelihood_obs0_forecast_t2_given_obs0_at_t1 = np.array([likelihood_obs_from_target_forecast_t2_given_obs0_at_t1[0], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs0_t2 = (posterior_t1 * likelihood_obs0_forecast_t2_given_obs0_at_t1) / (posterior_t1 * likelihood_obs0_forecast_t2_given_obs0_at_t1).sum()

    likelihood_obs1_forecast_t2_given_obs0_at_t1 = np.array([likelihood_obs_from_target_forecast_t2_given_obs0_at_t1[1], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs1_t2 = (posterior_t1 * likelihood_obs1_forecast_t2_given_obs0_at_t1) / (posterior_t1 * likelihood_obs1_forecast_t2_given_obs0_at_t1).sum()
    forecasted_S0 = (-posterior_forecast_given_obs0_t2 * np.log2(posterior_forecast_given_obs0_t2)).sum()
    forecasted_S1 = (-posterior_forecast_given_obs1_t2 * np.log2(posterior_forecast_given_obs1_t2)).sum()

    prob_obs0_at_forecast_t2_given_obs0_at_t1 = (posterior_t1 * np.array([likelihood_obs_from_target_forecast_t2_given_obs0_at_t1[0], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()
    prob_obs1_at_forecast_t2_given_obs0_at_t1 = (posterior_t1 * np.array([likelihood_obs_from_target_forecast_t2_given_obs0_at_t1[1], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()

    expected_forecast_t2_S = (forecasted_S0 * prob_obs0_at_forecast_t2_given_obs0_at_t1) + (forecasted_S1 * prob_obs1_at_forecast_t2_given_obs0_at_t1)
    expected_forecast_t2_S_per_step[i] = expected_forecast_t2_S

deltaS_t1 = current_entropyS_t1 - expected_forecast_t2_S_per_step

In [57]:
if (deltaS_t1==(deltaS_t1.max())).all():
    step_to_take = candidate_steps[1]
else:
    step_to_take = candidate_steps[deltaS_t1.argmax()]
step_to_take

np.float64(1.5707963267948966)

In [58]:
updated_angle = initial_angle + np.rad2deg(step_to_take)

if updated_angle >= 60 and updated_angle <= 120:
    code_received = 1
else:
    code_received = 0
updated_angle, code_received

(np.float64(90.0), 1)

In [60]:
state_transition_matrix = get_state_transition_matrix(step_to_take)

joint_prob_obs_state_t2 = (joint_prob_obs0_state_t1 @ state_transition_matrix) * EMISSION_MATRIX_B
joint_prob_obs1_state_t2 = joint_prob_obs_state_t2[code_received]
likelihood_obs1_from_target_t2 = joint_prob_obs1_state_t2.sum()

likelihood_obs1_from_target_given_obs0_at_t1 = likelihood_obs1_from_target_t2 / likelihood_obs0_from_target_t1
likelihood_obs1_given_obs0_at_t1 = np.array([likelihood_obs1_from_target_given_obs0_at_t1, LIKELIHOOD_OBS_FROM_NONTARGET])

posterior_given_obs1_t2 = (posterior_t1*likelihood_obs1_given_obs0_at_t1) / (posterior_t1*likelihood_obs1_given_obs0_at_t1).sum()
current_entropyS_t2 = (-posterior_given_obs1_t2*np.log2(posterior_given_obs1_t2)).sum()
current_entropyS_t2

np.float64(0.9447947327666398)

In [ ]:
candidate_steps = np.array([-np.pi/2, np.pi/2])
expected_forecast_t3_S_per_step = np.zeros(candidate_steps.shape[0])

In [ ]:
for i, forecast_t3_candidate_step in enumerate(candidate_steps):
    state_transition_matrix = get_state_transition_matrix(forecast_t3_candidate_step)

    joint_prob_obs_state_forecast_t3 = (joint_prob_obs1_state_t2 @ state_transition_matrix) * EMISSION_MATRIX_B
    likelihood_obs_from_target_forecast_t3 = joint_prob_obs_state_forecast_t3.sum(axis=1)
    likelihood_obs_from_target_forecast_t3_given_obs_until_t2 = likelihood_obs_from_target_forecast_t3 / likelihood_obs1_from_target_t2

    likelihood_obs0_forecast_t3_given_obs_until_t2 = np.array([likelihood_obs_from_target_forecast_t3_given_obs_until_t2[0], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs0_t3 = (posterior_given_obs1_t2 * likelihood_obs0_forecast_t3_given_obs_until_t2) / (posterior_given_obs1_t2 * likelihood_obs0_forecast_t3_given_obs_until_t2).sum()

    likelihood_obs1_forecast_t3_given_obs_until_t2 = np.array([likelihood_obs_from_target_forecast_t3_given_obs_until_t2[1], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs1_t3 = (posterior_given_obs1_t2 * likelihood_obs1_forecast_t3_given_obs_until_t2) / (posterior_given_obs1_t2 * likelihood_obs1_forecast_t3_given_obs_until_t2).sum()

    forecasted_S0 = (-posterior_forecast_given_obs0_t3 * np.log2(posterior_forecast_given_obs0_t3)).sum()
    forecasted_S1 = (-posterior_forecast_given_obs1_t3 * np.log2(posterior_forecast_given_obs1_t3)).sum()

    prob_obs0_at_forecast_t3_given_obs_until_t2 = (posterior_given_obs1_t2 * np.array([likelihood_obs_from_target_forecast_t3_given_obs_until_t2[0], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()
    prob_obs1_at_forecast_t3_given_obs_until_t2 = (posterior_given_obs1_t2 * np.array([likelihood_obs_from_target_forecast_t3_given_obs_until_t2[1], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()

    expected_forecast_t3_S = (forecasted_S0 * prob_obs0_at_forecast_t3_given_obs_until_t2) + (forecasted_S1 * prob_obs1_at_forecast_t3_given_obs_until_t2)
    expected_forecast_t3_S_per_step[i] = expected_forecast_t3_S

deltaS_t2 = current_entropyS_t2 - expected_forecast_t3_S_per_step
deltaS_t2

array([0.11397941, 0.08123557])

In [ ]:
if (deltaS_t2==(deltaS_t2.max())).all():
    step_to_take = candidate_steps[1]
else:
    step_to_take = candidate_steps[deltaS_t2.argmax()]
step_to_take

np.float64(-1.5707963267948966)

In [ ]:
updated_angle = updated_angle + np.rad2deg(step_to_take)

if updated_angle >= 60 and updated_angle <= 120:
    code_received = 1
else:
    code_received = 0
updated_angle, code_received

(np.float64(0.0), 0)

In [ ]:
state_transition_matrix = get_state_transition_matrix(step_to_take)

joint_prob_obs_state_t3 = (joint_prob_obs1_state_t2 @ state_transition_matrix) * EMISSION_MATRIX_B
joint_prob_obs0_state_t3 = joint_prob_obs_state_t3[code_received]
likelihood_obs0_from_target_t3 = joint_prob_obs0_state_t3.sum()

likelihood_obs0_from_target_given_obs_until_t2 = likelihood_obs0_from_target_t3 / likelihood_obs1_from_target_t2
likelihood_obs0_given_obs_until_t2 = np.array([likelihood_obs0_from_target_given_obs_until_t2, LIKELIHOOD_OBS_FROM_NONTARGET])

posterior_given_obs0_t3 = (posterior_given_obs1_t2*likelihood_obs0_given_obs_until_t2) / (posterior_given_obs1_t2*likelihood_obs0_given_obs_until_t2).sum()
current_entropyS_t3 = (-posterior_given_obs0_t3*np.log2(posterior_given_obs0_t3)).sum()
current_entropyS_t3

np.float64(0.9999999078178262)

In [ ]:
candidate_steps = np.array([-np.pi/2, np.pi/2])
expected_forecast_t4_S_per_step = np.zeros(candidate_steps.shape[0])

In [ ]:
for i, forecast_t4_candidate_step in enumerate(candidate_steps):
    state_transition_matrix = get_state_transition_matrix(forecast_t4_candidate_step)

    joint_prob_obs_state_forecast_t4 = (joint_prob_obs0_state_t3 @ state_transition_matrix) * EMISSION_MATRIX_B
    likelihood_obs_from_target_forecast_t4 = joint_prob_obs_state_forecast_t4.sum(axis=1)
    likelihood_obs_from_target_forecast_t4_given_obs_until_t3 = likelihood_obs_from_target_forecast_t4 / likelihood_obs0_from_target_t3

    likelihood_obs0_forecast_t4_given_obs_until_t3 = np.array([likelihood_obs_from_target_forecast_t4_given_obs_until_t3[0], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs0_t4 = (posterior_given_obs0_t3 * likelihood_obs0_forecast_t4_given_obs_until_t3) / (posterior_given_obs0_t3 * likelihood_obs0_forecast_t4_given_obs_until_t3).sum()

    likelihood_obs1_forecast_t4_given_obs_until_t3 = np.array([likelihood_obs_from_target_forecast_t4_given_obs_until_t3[1], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs1_t4 = (posterior_given_obs0_t3 * likelihood_obs1_forecast_t4_given_obs_until_t3) / (posterior_given_obs0_t3 * likelihood_obs1_forecast_t4_given_obs_until_t3).sum()

    forecasted_S0 = (-posterior_forecast_given_obs0_t4 * np.log2(posterior_forecast_given_obs0_t4)).sum()
    forecasted_S1 = (-posterior_forecast_given_obs1_t4 * np.log2(posterior_forecast_given_obs1_t4)).sum()

    prob_obs0_at_forecast_t4_given_obs_until_t3 = (posterior_given_obs0_t3 * np.array([likelihood_obs_from_target_forecast_t4_given_obs_until_t3[0], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()
    prob_obs1_at_forecast_t4_given_obs_until_t3 = (posterior_given_obs0_t3 * np.array([likelihood_obs_from_target_forecast_t4_given_obs_until_t3[1], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()

    expected_forecast_t4_S = (forecasted_S0 * prob_obs0_at_forecast_t4_given_obs_until_t3) + (forecasted_S1 * prob_obs1_at_forecast_t4_given_obs_until_t3)
    expected_forecast_t4_S_per_step[i] = expected_forecast_t4_S

deltaS_t3 = current_entropyS_t3 - expected_forecast_t4_S_per_step
deltaS_t3

array([0.20219164, 0.03067742])

In [ ]:
if (deltaS_t3==(deltaS_t3.max())).all():
    step_to_take = candidate_steps[1]
else:
    step_to_take = candidate_steps[deltaS_t3.argmax()]
step_to_take

np.float64(-1.5707963267948966)

In [ ]:
updated_angle = updated_angle + np.rad2deg(step_to_take)

if updated_angle >= 60 and updated_angle <= 120:
    code_received = 1
else:
    code_received = 0
updated_angle, code_received

(np.float64(-90.0), 0)

In [ ]:
state_transition_matrix = get_state_transition_matrix(step_to_take)

joint_prob_obs_state_t4 = (joint_prob_obs0_state_t3 @ state_transition_matrix) * EMISSION_MATRIX_B
joint_prob_obs0_state_t4 = joint_prob_obs_state_t4[code_received]
likelihood_obs0_from_target_t4 = joint_prob_obs0_state_t4.sum()

likelihood_obs0_from_target_given_obs_until_t3 = likelihood_obs0_from_target_t4 / likelihood_obs0_from_target_t3
likelihood_obs0_given_obs_until_t3 = np.array([likelihood_obs0_from_target_given_obs_until_t3, LIKELIHOOD_OBS_FROM_NONTARGET])

posterior_given_obs0_t4 = (posterior_given_obs0_t3*likelihood_obs0_given_obs_until_t3) / (posterior_given_obs0_t3*likelihood_obs0_given_obs_until_t3).sum()
current_entropyS_t4 = (-posterior_given_obs0_t4*np.log2(posterior_given_obs0_t4)).sum()
current_entropyS_t4

np.float64(0.9300034899902334)

In [ ]:
candidate_steps = np.array([-np.pi/2, np.pi/2])
expected_forecast_t5_S_per_step = np.zeros(candidate_steps.shape[0])

In [ ]:
for i, forecast_t5_candidate_step in enumerate(candidate_steps):
    state_transition_matrix = get_state_transition_matrix(forecast_t5_candidate_step)

    joint_prob_obs_state_forecast_t5 = (joint_prob_obs0_state_t4 @ state_transition_matrix) * EMISSION_MATRIX_B
    likelihood_obs_from_target_forecast_t5 = joint_prob_obs_state_forecast_t5.sum(axis=1)
    likelihood_obs_from_target_forecast_t5_given_obs_until_t4 = likelihood_obs_from_target_forecast_t5 / likelihood_obs0_from_target_t4

    likelihood_obs0_forecast_t5_given_obs_until_t4 = np.array([likelihood_obs_from_target_forecast_t5_given_obs_until_t4[0], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs0_t5 = (posterior_given_obs0_t4 * likelihood_obs0_forecast_t5_given_obs_until_t4) / (posterior_given_obs0_t4 * likelihood_obs0_forecast_t5_given_obs_until_t4).sum()

    likelihood_obs1_forecast_t5_given_obs_until_t4 = np.array([likelihood_obs_from_target_forecast_t5_given_obs_until_t4[1], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs1_t5 = (posterior_given_obs0_t4 * likelihood_obs1_forecast_t5_given_obs_until_t4) / (posterior_given_obs0_t4 * likelihood_obs1_forecast_t5_given_obs_until_t4).sum()

    forecasted_S0 = (-posterior_forecast_given_obs0_t5 * np.log2(posterior_forecast_given_obs0_t5)).sum()
    forecasted_S1 = (-posterior_forecast_given_obs1_t5 * np.log2(posterior_forecast_given_obs1_t5)).sum()

    prob_obs0_at_forecast_t5_given_obs_until_t4 = (posterior_given_obs0_t4 * np.array([likelihood_obs_from_target_forecast_t5_given_obs_until_t4[0], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()
    prob_obs1_at_forecast_t5_given_obs_until_t4 = (posterior_given_obs0_t4 * np.array([likelihood_obs_from_target_forecast_t5_given_obs_until_t4[1], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()

    expected_forecast_t5_S = (forecasted_S0 * prob_obs0_at_forecast_t5_given_obs_until_t4) + (forecasted_S1 * prob_obs1_at_forecast_t5_given_obs_until_t4)
    expected_forecast_t5_S_per_step[i] = expected_forecast_t5_S

deltaS_t4 = current_entropyS_t4 - expected_forecast_t5_S_per_step
deltaS_t4

array([0.04427535, 0.12753681])

In [ ]:
if (deltaS_t4==(deltaS_t4.max())).all():
    step_to_take = candidate_steps[1]
else:
    step_to_take = candidate_steps[deltaS_t4.argmax()]
step_to_take

np.float64(1.5707963267948966)

In [ ]:
updated_angle = updated_angle + np.rad2deg(step_to_take)

if updated_angle >= 60 and updated_angle <= 120:
    code_received = 1
else:
    code_received = 0
updated_angle, code_received

(np.float64(0.0), 0)

In [ ]:
state_transition_matrix = get_state_transition_matrix(step_to_take)

joint_prob_obs_state_t5 = (joint_prob_obs0_state_t4 @ state_transition_matrix) * EMISSION_MATRIX_B
joint_prob_obs0_state_t5 = joint_prob_obs_state_t5[code_received]
likelihood_obs0_from_target_t5 = joint_prob_obs0_state_t5.sum()

likelihood_obs0_from_target_given_obs_until_t4 = likelihood_obs0_from_target_t5 / likelihood_obs0_from_target_t4
likelihood_obs0_given_obs_until_t4 = np.array([likelihood_obs0_from_target_given_obs_until_t4, LIKELIHOOD_OBS_FROM_NONTARGET])

posterior_given_obs0_t5 = (posterior_given_obs0_t4*likelihood_obs0_given_obs_until_t4) / (posterior_given_obs0_t4*likelihood_obs0_given_obs_until_t4).sum()
current_entropyS_t5 = (-posterior_given_obs0_t5*np.log2(posterior_given_obs0_t5)).sum()
current_entropyS_t5

np.float64(0.7768289111851792)

In [ ]:
candidate_steps = np.array([-np.pi/2, np.pi/2])
expected_forecast_t6_S_per_step = np.zeros(candidate_steps.shape[0])

In [ ]:
for i, forecast_t6_candidate_step in enumerate(candidate_steps):
    state_transition_matrix = get_state_transition_matrix(forecast_t6_candidate_step)

    joint_prob_obs_state_forecast_t6 = (joint_prob_obs0_state_t5 @ state_transition_matrix) * EMISSION_MATRIX_B
    likelihood_obs_from_target_forecast_t6 = joint_prob_obs_state_forecast_t6.sum(axis=1)
    likelihood_obs_from_target_forecast_t6_given_obs_until_t5 = likelihood_obs_from_target_forecast_t6 / likelihood_obs0_from_target_t5

    likelihood_obs0_forecast_t6_given_obs_until_t5 = np.array([likelihood_obs_from_target_forecast_t6_given_obs_until_t5[0], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs0_t6 = (posterior_given_obs0_t5 * likelihood_obs0_forecast_t6_given_obs_until_t5) / (posterior_given_obs0_t5 * likelihood_obs0_forecast_t6_given_obs_until_t5).sum()

    likelihood_obs1_forecast_t6_given_obs_until_t5 = np.array([likelihood_obs_from_target_forecast_t6_given_obs_until_t5[1], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs1_t6 = (posterior_given_obs0_t5 * likelihood_obs1_forecast_t6_given_obs_until_t5) / (posterior_given_obs0_t5 * likelihood_obs1_forecast_t6_given_obs_until_t5).sum()

    forecasted_S0 = (-posterior_forecast_given_obs0_t6 * np.log2(posterior_forecast_given_obs0_t6)).sum()
    forecasted_S1 = (-posterior_forecast_given_obs1_t6 * np.log2(posterior_forecast_given_obs1_t6)).sum()

    prob_obs0_at_forecast_t6_given_obs_until_t5 = (posterior_given_obs0_t5 * np.array([likelihood_obs_from_target_forecast_t6_given_obs_until_t5[0], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()
    prob_obs1_at_forecast_t6_given_obs_until_t5 = (posterior_given_obs0_t5 * np.array([likelihood_obs_from_target_forecast_t6_given_obs_until_t5[1], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()

    expected_forecast_t6_S = (forecasted_S0 * prob_obs0_at_forecast_t6_given_obs_until_t5) + (forecasted_S1 * prob_obs1_at_forecast_t6_given_obs_until_t5)
    expected_forecast_t6_S_per_step[i] = expected_forecast_t6_S

deltaS_t5 = current_entropyS_t5 - expected_forecast_t6_S_per_step
deltaS_t5

array([0.14514375, 0.02558569])

In [ ]:
if (deltaS_t5==(deltaS_t5.max())).all():
    step_to_take = candidate_steps[1]
else:
    step_to_take = candidate_steps[deltaS_t5.argmax()]
step_to_take

np.float64(-1.5707963267948966)

In [ ]:
updated_angle = updated_angle + np.rad2deg(step_to_take)

if updated_angle >= 60 and updated_angle <= 120:
    code_received = 1
else:
    code_received = 0
updated_angle, code_received

(np.float64(-90.0), 0)

In [ ]:
state_transition_matrix = get_state_transition_matrix(step_to_take)

joint_prob_obs_state_t6 = (joint_prob_obs0_state_t5 @ state_transition_matrix) * EMISSION_MATRIX_B
joint_prob_obs0_state_t6 = joint_prob_obs_state_t6[code_received]
likelihood_obs0_from_target_t6 = joint_prob_obs0_state_t6.sum()

likelihood_obs0_from_target_given_obs_until_t5 = likelihood_obs0_from_target_t6 / likelihood_obs0_from_target_t5
likelihood_obs0_given_obs_until_t5 = np.array([likelihood_obs0_from_target_given_obs_until_t5, LIKELIHOOD_OBS_FROM_NONTARGET])

posterior_given_obs0_t6 = (posterior_given_obs0_t5*likelihood_obs0_given_obs_until_t5) / (posterior_given_obs0_t5*likelihood_obs0_given_obs_until_t5).sum()
current_entropyS_t6 = (-posterior_given_obs0_t6*np.log2(posterior_given_obs0_t6)).sum()
current_entropyS_t6

np.float64(0.5781562917283571)

In [ ]:
candidate_steps = np.array([-np.pi/2, np.pi/2])
expected_forecast_t7_S_per_step = np.zeros(candidate_steps.shape[0])

In [ ]:
for i, forecast_t7_candidate_step in enumerate(candidate_steps):
    state_transition_matrix = get_state_transition_matrix(forecast_t7_candidate_step)

    joint_prob_obs_state_forecast_t7 = (joint_prob_obs0_state_t6 @ state_transition_matrix) * EMISSION_MATRIX_B
    likelihood_obs_from_target_forecast_t7 = joint_prob_obs_state_forecast_t7.sum(axis=1)
    likelihood_obs_from_target_forecast_t7_given_obs_until_t6 = likelihood_obs_from_target_forecast_t7 / likelihood_obs0_from_target_t6

    likelihood_obs0_forecast_t7_given_obs_until_t6 = np.array([likelihood_obs_from_target_forecast_t7_given_obs_until_t6[0], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs0_t7 = (posterior_given_obs0_t6 * likelihood_obs0_forecast_t7_given_obs_until_t6) / (posterior_given_obs0_t6 * likelihood_obs0_forecast_t7_given_obs_until_t6).sum()

    likelihood_obs1_forecast_t7_given_obs_until_t6 = np.array([likelihood_obs_from_target_forecast_t7_given_obs_until_t6[1], LIKELIHOOD_OBS_FROM_NONTARGET])
    posterior_forecast_given_obs1_t7 = (posterior_given_obs0_t6 * likelihood_obs1_forecast_t7_given_obs_until_t6) / (posterior_given_obs0_t6 * likelihood_obs1_forecast_t7_given_obs_until_t6).sum()

    forecasted_S0 = (-posterior_forecast_given_obs0_t7 * np.log2(posterior_forecast_given_obs0_t7)).sum()
    forecasted_S1 = (-posterior_forecast_given_obs1_t7 * np.log2(posterior_forecast_given_obs1_t7)).sum()

    prob_obs0_at_forecast_t7_given_obs_until_t6 = (posterior_given_obs0_t6 * np.array([likelihood_obs_from_target_forecast_t7_given_obs_until_t6[0], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()
    prob_obs1_at_forecast_t7_given_obs_until_t6 = (posterior_given_obs0_t6 * np.array([likelihood_obs_from_target_forecast_t7_given_obs_until_t6[1], LIKELIHOOD_OBS_FROM_NONTARGET])).sum()

    expected_forecast_t7_S = (forecasted_S0 * prob_obs0_at_forecast_t7_given_obs_until_t6) + (forecasted_S1 * prob_obs1_at_forecast_t7_given_obs_until_t6)
    expected_forecast_t7_S_per_step[i] = expected_forecast_t7_S

deltaS_t6 = current_entropyS_t6 - expected_forecast_t7_S_per_step
deltaS_t6

array([0.02291352, 0.07681876])

In [ ]:
if (deltaS_t6==(deltaS_t6.max())).all():
    step_to_take = candidate_steps[1]
else:
    step_to_take = candidate_steps[deltaS_t6.argmax()]
step_to_take

np.float64(1.5707963267948966)

In [ ]:
updated_angle = updated_angle + np.rad2deg(step_to_take)

if updated_angle >= 60 and updated_angle <= 120:
    code_received = 1
else:
    code_received = 0
updated_angle, code_received

(np.float64(0.0), 0)

In [ ]:
state_transition_matrix = get_state_transition_matrix(step_to_take)

joint_prob_obs_state_t7 = (joint_prob_obs0_state_t6 @ state_transition_matrix) * EMISSION_MATRIX_B
joint_prob_obs0_state_t7 = joint_prob_obs_state_t7[code_received]
likelihood_obs0_from_target_t7 = joint_prob_obs0_state_t7.sum()

likelihood_obs0_from_target_given_obs_until_t6 = likelihood_obs0_from_target_t7 / likelihood_obs0_from_target_t6
likelihood_obs0_given_obs_until_t6 = np.array([likelihood_obs0_from_target_given_obs_until_t6, LIKELIHOOD_OBS_FROM_NONTARGET])

posterior_given_obs0_t7 = (posterior_given_obs0_t6*likelihood_obs0_given_obs_until_t6) / (posterior_given_obs0_t6*likelihood_obs0_given_obs_until_t6).sum()
current_entropyS_t7 = (-posterior_given_obs0_t7*np.log2(posterior_given_obs0_t7)).sum()
current_entropyS_t7

np.float64(0.4090386372424645)

In [ ]:
PRIOR_TARGET_PROB, posterior_t1, posterior_given_obs1_t2, posterior_given_obs0_t3, posterior_given_obs0_t4, posterior_given_obs0_t5, posterior_given_obs0_t6, posterior_given_obs0_t7

(array([0.5, 0.5]),
 array([0.625, 0.375]),
 array([0.36256866, 0.63743134]),
 array([0.49982126, 0.50017874]),
 array([0.65447976, 0.34552024]),
 array([0.77067721, 0.22932279]),
 array([0.86231027, 0.13768973]),
 array([0.91804269, 0.08195731]))